# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

## Text Preprocessing
### reference
> * [Google guide](https://developers.google.com/machine-learning/guides/text-classification/step-3)
> * N-grams
>> * [scikit-learn working with text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#)
>> * [scikit-learn text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
>> * [한글 자료](https://datascienceschool.net/03%20machine%20learning/03.01.03%20Scikit-Learn%EC%9D%98%20%EB%AC%B8%EC%84%9C%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EB%8A%A5.html)
> * Sequence
>> * [keras text classification](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [tensorflow text classification](https://www.tensorflow.org/tutorials/keras/text_classification)

---
#### 

### 0. 라이브러리 설치 및 불러오기

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 54.0 MB/s 


In [5]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[]
[]


### 1.  데이터 가져오기

In [7]:
# 데이터를 가져옵니다.
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KT_mini_4/spam.csv')

In [8]:
data.head()

,Unnamed: 0,text,label
0,1,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,spam
1,2,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],spam
2,3,[Web발신][대//박]가(.원)입코(WN)드MEP.com,spam
3,4,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,spam
4,5,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,spam


#### 1-1. processing label

In [9]:
# label 데이터를 수치형으로 변환합니다.
from sklearn import preprocessing

# spam은 1, ham은 0
le = preprocessing.LabelEncoder()
data['label'] = le.fit_transform(data['label'].values)

In [10]:
data.tail()

,Unnamed: 0,text,label
20095,20096,"[Web발신]이상투자황기봉님""이재훈""투자컨설담당자배정완료.담당자확인▼http://b...",1
20096,20097,[Web발신](광고)조마?峙???충?矛?%:까지 ↓주소Joma.카JM톡무료거부,1
20097,20098,[Web발신]김환택님지금이라도늦지않았습니다.월긴급정보http://pf.kakao.c...,1
20098,20099,[Web발신]년노하우명품구매대행논스토리nonstory.com항상아낌없는사랑과성원에감...,0
20099,20100,[Web발신]최현님월일특급정보!https://pf.kakao.com/_SfcsT,1


In [11]:
# 결측치 제거
data.dropna(axis=0, inplace=True)
data.isnull().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [12]:
data

,Unnamed: 0,text,label
0,1,[Web발신]박춘규회원님손절주식은그만월급배만드는법http://lco.jp/eA,1
1,2,[Web발신]골든브릿지에서손실을보셨나요??http://bitly.kr/bRGtq[FW],1
2,3,[Web발신][대//박]가(.원)입코(WN)드MEP.com,1
3,4,[Web발신](광고)이정미님아직도주식하시나요http://pf.kakao.com/_u...,1
4,5,[Web발신]송승용님사람이모이는곳에는이유가있습니다하루~정보공개is.gd/JsJP,1
...,...,...,...
20095,20096,"[Web발신]이상투자황기봉님""이재훈""투자컨설담당자배정완료.담당자확인▼http://b...",1
20096,20097,[Web발신](광고)조마?峙???충?矛?%:까지 ↓주소Joma.카JM톡무료거부,1
20097,20098,[Web발신]김환택님지금이라도늦지않았습니다.월긴급정보http://pf.kakao.c...,1
20098,20099,[Web발신]년노하우명품구매대행논스토리nonstory.com항상아낌없는사랑과성원에감...,0


### 2. Train Validation(Test) Split

In [13]:
# train validation set으로 분리합니다.
x = data['text']
y = data['label']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, random_state=2022, test_size=0.25)

### 3. Vectorize texts

#### 3-1. N-grams Vectorize [참고](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#extracting-features-from-text-files)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# train
x_train_ngrams = vectorizer.fit_transform(x_train).toarray()

# valid
x_valid_ngrams = vectorizer.transform(x_valid).toarray()

In [15]:
x_train_ngrams.shape, x_valid_ngrams.shape

((15066, 27913), (5023, 27913))

#### 3-2. Sequence Vectorize [참고](https://developers.google.com/machine-learning/guides/text-classification/step-3)

In [19]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text

# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated.
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(x_train) # 사전을 만든다 (매핑까지 하는 상태)

    # Vectorize training and validation texts.
    x_train_seq = tokenizer.texts_to_sequences(x_train)
    x_val_seq = tokenizer.texts_to_sequences(x_valid)

    # Get max sequence length.
    max_length = len(max(x_train_seq, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train_seq = sequence.pad_sequences(x_train_seq, maxlen=max_length)
    x_val_seq = sequence.pad_sequences(x_val_seq, maxlen=max_length)
    return x_train_seq, x_val_seq, tokenizer.word_index # 출력해보자

In [20]:
x_train_seq, x_valid_seq, word_idx = sequence_vectorize(x_train, x_valid)

In [21]:
x_train_seq.shape, x_valid_seq.shape

((15066, 23), (5023, 23))

In [22]:
len(word_idx)

27701

### 4. Save data

#### 4-1. Save N-grams

In [30]:
import pickle

In [31]:
# N-grams 방식으로 vectorize한 데이터의 shape을 확인해봅니다.
x_train_ngrams.shape, x_valid_ngrams.shape

((15066, 27913), (5023, 27913))

In [32]:
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
with open("x_train_ngrams.pickle","wb") as f:
    pickle.dump(x_train_ngrams, f) 
    
with open("x_valid_ngrams.pickle","wb") as f:
    pickle.dump(x_valid_ngrams, f) 

#### 4-2. Save sequence

In [33]:
# Sequence 형태로 vectorize한 데이터의 shape을 확인해봅니다.
x_train_seq.shape, x_valid_seq.shape

((15066, 23), (5023, 23))

In [34]:
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
# 모델 학습시에 활용 가능하도록 전처리 데이터를 저장해보도록 하겠습니다.
with open("x_train_seq.pickle","wb") as f:
    pickle.dump(x_train_seq, f) 
    
with open("x_valid_seq.pickle","wb") as f:
    pickle.dump(x_valid_seq, f) 

#### 4-3. Save label

In [35]:
# label 데이터의 shape을 확인하고 저장합니다.
y_train.shape, y_valid.shape

((15066,), (5023,))

In [36]:
with open("y_train.pickle","wb") as f:
    pickle.dump(y_train, f) 
    
with open("y_valid.pickle","wb") as f:
    pickle.dump(y_valid, f) 